<a href="https://colab.research.google.com/github/MobinaSedaghat/AminoAcid/blob/main/AminoAcidCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Functions

## Search

In [ ]:
import urllib.request
def pdb_search(pdb):
  with urllib.request.urlopen('https://www.rcsb.org/fasta/entry/{}/display'.format(pdb)) as response:
    html = response.read()


  data = str(html[:-1]).split('\\n')

  return data

## Calculation

In [ ]:
import itertools
def amino_charge():

  amino_acid = {'A' : 'alanine', 'P' : 'proline', 'Q' : 'glutamine', 'C' : 'cystine','R' : 'arginine','D' : 'aspartate','S' : 'serine','E' : 'glutamatic','T' : 'threonine','F' : 'phenylalanine'
  ,'G' : 'glycine','V': 'valine','H' : 'histidine','W' : 'tryptophan', 'I' : 'isoleucine' ,'Y' : 'tyrosine','K' :'lysine', 'L' : 'leucine' , 'M' : 'methionine'
    ,'N' : 'asparagine'}

  charge = {('alanine','proline','glycine','valine','isoleucine','leucine','methionine') : 'None Polar' , ('phenylalanine','tryptophan','tyrosine') : 'Aromatic', ('serine','threonine','cystine','asparagine','glutamine'): 'Polar',
            ('lysine','arginine','histidine'):'Positively charged', ('aspartate','glutamatic'):'Negatively charged'}
  charges = {}
  for k, v in charge.items():
      for key in k:
          charges[key] = v
  my_dict = {}
  my_list1 = []
  inv_charges = {}
  for k in charge.keys():
      inv_charges[charge[k]] = k
  x=list(zip(inv_charges.keys() , inv_charges.values()))
  for i in range(len(x)):
    temp = []
    for j in range(len(x[i][1])):
      temp.append( x[i][1][j])
    my_dict [x[i][0]] = temp
  for i in range (len(my_dict.keys())):
    a=[list(my_dict.keys())[i]]
    b=list(my_dict.values())[i]
    my_list1 = my_list1 + list(itertools.product(a,b))
  return amino_acid,charge,charges,my_list1

In [ ]:
def count_aminos(amino_acid):
  amino_list = []
  for i in range (len(data)):
    if i%2==0:
      amino_list.append({x : 0 for x in amino_acid.values()})
      continue
    for j in data[i][:-1]:
      amino_list[int(i/2)][amino_acid[j]] = amino_list[int(i/2)][amino_acid[j]] + 1

  count_charges(amino_list)


In [ ]:
def count_charges(amino_list):
  int_charges = []
  charge_list = []
  amino_count=[[]]*len(amino_list)
  charges_list_dict = [[]]*len(amino_list)

  for i in range (len(amino_list)):
    charge_list.append({x : 0 for x in charges.values()})
    amino_count = list(amino_list[i].items())
    charges_list_dict[i] =  list(charge_list[i].items())
    charges_list_dict[i] = [list(i) for i in charges_list_dict[i]]

    for j in amino_list[i]:
      x = [a[0] for a in charges_list_dict[i]].index(charges[j])
      charges_list_dict[i][x][1] = charges_list_dict[i][x][1] + amino_count[amino_count.index((j,amino_list[i][j]))][1]

    int_charges .append([charges_list_dict[i][t][1] for t in range (len(charges_list_dict[i]))])

  print_results(amino_list,charges_list_dict, int_charges)

In [ ]:
def print_results(amino_list,charges_list_dict, int_charges):
  total = 0
  total_chain = []
  perc = []
  perc_chain=[]
  total_perc=[] * int(len(data)/2)

  for i in range(int(len(data)/2)):
    total = 0
    print("\n  \n" + data[(i*2)] + " --> \n")
    for j in range(len(charges_list_dict[i])):

      total =  charges_list_dict[i][j][1] + total
    total_chain.append(total)
    for j in range(len(charges_list_dict[i])):
      perc.append(str(round(charges_list_dict[i][j][1]/total*100,3)) + " %")

      perc_chain = perc_chain + [round((charges_list_dict[i][j][1]/total*100),3)]
      print(charges_list_dict[i][j][0] + " : " + str(perc_chain[j]) + " %")

    total_perc =total_perc + [perc_chain]
    perc_chain = []

  final_list(amino_list,total_chain, total_perc,int_charges,charges_list_dict)


## Visualization




In [ ]:

def final_list(amino_list,total_chain, total_perc,int_charges,charges_list_dict):
  a =[]
  t = []
  my_list2 = []
  res=[]
  for j in range (int(len(data)/2)):
    m= tuple()
    for i in range (len(my_list1)):
      m = amino_list[j][my_list1[i][1]]
      t.append(m)
      a.append([my_list1[i][0],my_list1[i][1] , m])
      m= tuple()
    res.append(t)
    t= []
    my_list2.append(a)
  data_frame(amino_list,my_list2,res,total_chain, total_perc,int_charges,charges_list_dict)


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
def data_frame(amino_list,my_list2,res,total_chain, total_perc,int_charges,charges_list_dict):
  col_list = []
  row_list1 = pd.MultiIndex.from_tuples(my_list1)
  for i in range (int(len(data)/2)):
    col_list = col_list + ['Chain '+ str(i+1)]
  df = pd.DataFrame(np.array(res).T, row_list1 , col_list)

  amino_acid_total=[]
  for i in range(len(amino_acid)):
    sum = 0
    for j in res:
      sum =  j[i]+ sum
    amino_acid_total.append(sum)
  df["total"] = amino_acid_total
  display(df)






  row_list2 = list(charge.values())
  print("\n \n")
  df2 = pd.DataFrame(np.array(int_charges).T, row_list2 , col_list)

  charge_total=[]
  for i in range(len(charge.values())):
    sum = 0
    for j in int_charges:
      sum =  j[i]+ sum
    charge_total.append(sum)

  df2["total"] = charge_total


  display(df2)
  print("\n \n")
  plot_bar(amino_list,res,col_list,my_list2,total_chain, total_perc,charges_list_dict)
  plot_pie(amino_acid_total,charge_total)



In [ ]:
import plotly.graph_objects as go
def plot_bar(amino_list,res,col_list,my_list2,total_chain, total_perc,charges_list_dict):
  x1 = []
  y1 = []
  x= []
  pe2 = []
  pe1 = []
  for i in range(2):
    l = [list(zip(*my_list1))[i]]
    l = [item for t in l for item in t]
    x.append(l)
  fig = go.Figure()

  for i in range(len(total_chain)):
    pe1.append( list(round(res[i][x]/total_chain[i] * 100 ,3)   for x in range (len(res[i]))))

  for i in range(len(total_chain)):
    fig.add_bar(name='chain ' + str(i+1) , x=x , y=res[i] , text = pe1[i], textposition="inside",textangle=0 )
  fig.update_traces( marker_line_color='rgb(8,48,107)',marker_line_width=1.5, )
  fig.update_layout(barmode="relative" )
  fig.show()



  for i in charges_list_dict:
    for j in range (len(i)):
      x1 = x1 + [i[j][0]]
      y1 = y1 + [i[j][1]]


  fig2 = go.Figure()

  for i in range(len(col_list)):
    ni2 = list(round(total_perc[i][x] ,3)   for x in range(len(total_perc[i])))
    pe2.append(list("{} %" .format(n) for n in ni2))

    fig2.add_bar(name='chain ' + str(i+1) , x = x1[:5] , y=y1, text = pe2[i], textposition="inside")
  fig2.update_traces( marker_line_color='rgb(8,48,107)',
                    marker_line_width=1.5, )
  fig2.update_layout(barmode="relative" )

  fig2.show()

In [ ]:
import plotly.express as px
def plot_pie(amino_acid_total,charge_total):
  fig = px.pie(values=amino_acid_total, names=amino_acid.values())
  fig.update_traces( marker_line_color='rgb(8,48,107)',
                    marker_line_width=1.5, )
  fig.show()

  fig2 = px.pie(values=charge_total, names=charge.values())
  fig2.update_traces( marker_line_color='rgb(8,48,107)',
                    marker_line_width=1.5, )
  fig2.show()

# Main

In [ ]:
try:
  pdb = input("Enter the PDB ID of the protein: ")  #'6P9X'
  data = pdb_search(pdb)
  amino_acid,charge,charges,my_list1 = amino_charge()
  count_aminos(amino_acid)

except:
  print("Not found!")

Enter the PDB ID of the protein: 6P9X

  
b'>6P9X_2|Chain B|Guanine nucleotide-binding protein G(I)/G(S)/G(T) subunit beta-1|Homo sapiens (9606) --> 

None Polar : 39.255 %
Aromatic : 7.736 %
Polar : 28.653 %
Positively charged : 13.181 %
Negatively charged : 11.175 %

  
>6P9X_3|Chain C[auth G]|Guanine nucleotide-binding protein G(I)/G(S)/G(O) subunit gamma-2|Homo sapiens (9606) --> 

None Polar : 45.714 %
Aromatic : 5.714 %
Polar : 20.0 %
Positively charged : 15.714 %
Negatively charged : 12.857 %

  
>6P9X_4|Chain D[auth N]|Nanobody 35|Lama glama (9844) --> 

None Polar : 36.496 %
Aromatic : 10.219 %
Polar : 32.847 %
Positively charged : 13.139 %
Negatively charged : 7.299 %

  
>6P9X_5|Chain E[auth R]|Corticotropin-releasing factor receptor 1|Homo sapiens (9606) --> 

None Polar : 42.488 %
Aromatic : 14.319 %
Polar : 23.005 %
Positively charged : 13.615 %
Negatively charged : 6.573 %

  
>6P9X_1|Chain A|Guanine nucleotide-binding protein G(s) subunit alpha isoforms short|Homo sapie

Chain 1  Chain 2  Chain 3  Chain 4  Chain 5  \
None Polar         alanine             30       13        9       27       26   
                   proline              5        4        6       16       14   
                   glycine             29        0       13       22       18   
                   valine              17        3        9       37       26   
                   isoleucine          18        4        3       30       23   
                   leucine             30        5        8       42       29   
                   methionine           8        3        2        7        7   
Aromatic           phenylalanine       11        3        5       29       20   
                   tryptophan           8        0        2       12        4   
                   tyrosine             8        1        7       20       14   
Polar              serine              30        4       16       29       18   
                   threonine           28        2       11       21       17   
                   cystine             14        2        4       13        8   
                   asparagine          16        4        5       21       23   
                   glutamine           12        2        9       14       21   
Positively charged lysine              10        7        5       14       26   
                   arginine            22        3        7       26       28   
                   histidine           14        1        6       18        7   
Negatively charged aspartate           30        3        5       12       31   
                   glutamatic           9        6        5       16       33   

                                  Chain 6  total  
None Polar         alanine              4    109  
                   proline              2     47  
                   glycine              0     82  
                   valine               1     93  
                   isoleucine           3     81  
                   leucine              7    121  
                   methionine           2     29  
Aromatic           phenylalanine        1     69  
                   tryptophan           0     26  
                   tyrosine             0     50  
Polar              serine               3    100  
                   threonine            1     80  
                   cystine              0     41  
                   asparagine           1     70  
                   glutamine            3     61  
Positively charged lysine               1     63  
                   arginine             3     89  
                   histidine            2     48  
Negatively charged aspartate            1     82  
                   glutamatic           6     75

,Chain 1,Chain 2,Chain 3,Chain 4,Chain 5,Chain 6,total
None Polar,137,32,50,181,143,19,562
Aromatic,27,4,14,61,38,1,145
Polar,100,14,45,98,87,8,352
Positively charged,46,11,18,58,61,6,200
Negatively charged,39,9,10,28,64,7,157
